In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import random
import re
from tqdm import tqdm
import numpy as np
import json

In [2]:
df = pd.read_csv('../data/original_convfinqa_dataset.csv')

In [3]:
df.shape

(1588, 2)

In [4]:
df.head()

,html_content,questions
0,"<html>\n <body>\n ""distribution date"" ) . unt...",['what was the percentage of the total common ...
1,"<html>\n <body>\n ""three factor formula"" ) . ...","['in the year of 2013, during the second quart..."
2,"<html>\n <body>\n $ 25.7 million in cash , in...","['in the year of 2009, how much did the income..."
3,<html>\n <body>\n $ 43.3 million in 2011 comp...,['what was the difference between the net sale...
4,<html>\n <body>\n ( $ 125 million ) and highe...,['what were north american industrial packagin...


In [5]:
df['index'] = df.index + 1

In [6]:
df = df.drop('questions', axis='columns')

In [7]:
df_index_info = pd.read_csv('../data/index_info.csv')

In [8]:
index_info = df_index_info['Index of data without headers'].to_list()

In [9]:
def beautify_html(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    table = soup.find('table')
    return table

In [10]:
df_with_added_rows = pd.DataFrame(columns=['index', 'html_content', 'question', 'answer'])
for index, df_rows in tqdm(df.iterrows()):
    table = beautify_html(df_rows['html_content'])
    rows = table.find_all('tr')
    if df_rows['index'] in index_info:
        # Split the HTML string by <td> tags
        random_row = random.randint(0, len(rows) - 1)
        
    else:
        random_row = random.randint(1, len(rows) - 1)
        
    cells = rows[random_row].find_all('td')
    data = [cell.get_text(strip=True) for cell in cells]
    
    question = "What is the value for " + data[1] + " ?"
    answers = [str(data[i]) for i in range(2, len(data))]
    
    # data[1] = data[1] + '_v1'
    for i in range(2, len(data)):
        random_chosen_value = str(random.randint(10, 999999))
        if np.random.uniform() < 0.5:
            random_chosen_value = '$ ' + random_chosen_value
            
        data[i] = random_chosen_value
        answers.append(random_chosen_value)
    
    str_data = '<tr> '
    for value in data:
        str_data += '<td> ' + str(value) + ' </td> '
        
    str_data += '</tr>'
    
    if df_rows['index'] in index_info:
        # Split the HTML string by <td> tags
        rows.insert(random.randint(0, len(rows)-1), str_data)
        
    else:
        rows.insert(random.randint(1, len(rows)-1), str_data)
    
    # Split the text based on <table> tags
    html_parts = df_rows['html_content'].split('<table')

    # Add back the <table> tag to the end of each part except the last one
    html_parts = [part + '<table>' if i < len(html_parts) - 1 else part for i, part in enumerate(html_parts)]

    str_row = ''  
    for row in rows:
        str_row += str(row)
    
    if len(html_parts) == 3:
        final_html_value = html_parts[0] + str_row + html_parts[2]
    else:
        final_html_value = html_parts[0] + str_row + ' </table> </body> </html>'
    df_with_added_rows.loc[len(df_with_added_rows)] = {"index": df_rows['index'],
            'html_content': final_html_value,
            'question': question,
            'answer': answers}
    # print(question)
    # print(answers)
    # print('*****')
    # if index == 2:
    #     print(final_html_value)
        # break

0it [00:00, ?it/s]

1588it [00:04, 349.25it/s]


In [12]:
def convert_table_to_pseudo_json(row):
    # Parse HTML content
    soup = BeautifulSoup(row['html_content'], 'html.parser')

    # Find the table
    table = soup.find('table')

    if int(row['index']) in index_info:
        lst = []
        for rows in table.find_all('tr'):
            row = rows.find_all('td')[1:]
            nlst = [] 
            for cell in row:
                nlst.append(cell.get_text().strip())
            lst.append(nlst)
            
        str_lst = str(lst).replace('[', '{').replace(']', '}')
        
        # Convert the dictionary to a string
        table_str = str(str_lst)
   
        # Convert the tag object to a string
        tag_str = str(table)
   
        # Concatenate the table string with the remaining HTML content
        output_html = str(soup).replace(tag_str, table_str)
        # output_html = html_content.replace('</body>', f'{table_str} .\n </body>')
        
    else: 
        # Extract data from the table
        rows = table.find_all('tr')
        header_row = rows[0]
        headers = [header.text.strip() for header in header_row.find_all('td')[1:]]  # Exclude the first column
        
        results = [] 
        for row in rows[1:]:
            columns = row.find_all('td')
            category = columns[1].text.strip()
            result = []
            for header, column in zip(headers[1:], columns[2:]):
                result.append([header, column.text.strip()])
            results.append([category, result])
        
        str_results = str(results).replace('[', '{').replace(']', '}')
        
        # Convert the dictionary to a string
        table_str = str(str_results)
   
        # Convert the tag object to a string
        tag_str = str(table)
   
        # Concatenate the table string with the remaining HTML content
        output_html = str(soup).replace(tag_str, table_str)
        # output_html = html_content.replace('</body>', f'{table_str} .\n </body>')
    return output_html 

In [13]:
for i, row in tqdm(df_with_added_rows.iterrows()):
    output = convert_table_to_pseudo_json(row)
    df_with_added_rows.at[i, 'pseudo_json_encoding'] = output

1588it [00:04, 367.06it/s]


In [14]:
for index, row in df_with_added_rows.iterrows():
    print(index)
    print(row['pseudo_json_encoding'])
    print(row['question'])
    print(row['answer'])
    print('***')
    if index == 20:
        break

0
<html>
<body>
  "distribution date" ) . until the distribution date ( or earlier redemption or expiration of the rights ) , the rights will be traded with , and only with , the common stock . until a right is exercised , the right will not entitle the holder thereof to any rights as a stockholder . if any person or group becomes an acquiring person , each holder of a right , other than rights beneficially owned by the acquiring person , will thereafter have the right to receive upon exercise and payment of the purchase price that number of shares of common stock having a market value of two times the purchase price and , if the company is acquired in a business combination transaction or 50% ( 50 % ) or more of its assets are sold , each holder of a right will thereafter have the right to receive upon exercise and payment of the purchase price that number of shares of common stock of the acquiring company which at the time of the transaction will have a market value of two times the 

In [15]:
df_with_added_rows.columns

Index(['index', 'html_content', 'question', 'answer', 'pseudo_json_encoding'], dtype='object')

In [16]:
odf = pd.read_csv('../data/pseudo_json_dataset_V1.csv')

In [17]:
merged_df = pd.concat([odf, df_with_added_rows], ignore_index=True)
merged_df.shape

(19200, 6)

In [18]:
merged_df.columns

Index(['Unnamed: 0', 'answer', 'html_content', 'index', 'question',
       'pseudo_json_encoding'],
      dtype='object')

In [19]:
merged_df = merged_df.drop('Unnamed: 0', axis='columns')

In [20]:
merged_df.shape

(19200, 5)

In [21]:
merged_df.columns

Index(['answer', 'html_content', 'index', 'question', 'pseudo_json_encoding'], dtype='object')

In [24]:
merged_df = merged_df.sample(frac=1).reset_index(drop=True)

In [25]:
merged_df.shape

(19200, 5)

In [26]:
merged_df.columns

Index(['answer', 'html_content', 'index', 'question', 'pseudo_json_encoding'], dtype='object')

In [27]:
merged_df.head()

,answer,html_content,index,question,pseudo_json_encoding
0,1.5% ( 1.5 % ),<html>\n <body>\n visa inc . notes to consoli...,1534,What is risk-free rate of return ( 2 ) for 2015 ?,<html>\n<body>\n visa inc . notes to consolid...
1,3568,<html>\n <body>\n notes to consolidated finan...,988,What is credit funds2 for unfunded commitments ?,<html>\n<body>\n notes to consolidated financ...
2,28649,"<html>\n <body>\n hologic , inc . notes to co...",618,What is net income for 2006 ?,"<html>\n<body>\n hologic , inc . notes to con..."
3,$21,<html>\n <body>\n corporate/other corporate/o...,373,What is net credit losses for 2018 ?,<html>\n<body>\n corporate/other corporate/ot...
4,7450.4,<html>\n <body>\n page 31 of 98 additional de...,1077,What is purchase obligations ( c ) for payment...,<html>\n<body>\n page 31 of 98 additional det...


In [28]:
merged_df.to_csv('../data/pseudo_json_dataset_V2.csv', index=False)